In [0]:
from google.colab import drive
import torch
drive.mount('/content/gdrive', force_remount=True)
print(torch.cuda.is_available())

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm # Displays a progress bar
import os
import torch
from sklearn.model_selection import train_test_split
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, Subset, DataLoader, random_split, sampler
from torch.optim import lr_scheduler
from PIL import Image
import time
import copy

In [0]:
ls '/content/gdrive/Shared drives/eecs442-project/arc10'

In [0]:
ls '/content/gdrive/Shared drives/eecs442-project/arc25'

# **Load Data for ResNet**

In [0]:
def LoadTrainDataSet(train_dir, batch_size, 
                     train_transform, valid_transform, validation_size=0.2):
    print("Loading Train Data ...")
    
    train_data = datasets.ImageFolder(root=train_dir, transform=train_transform)
    valid_data = datasets.ImageFolder(root=train_dir, transform=valid_transform)

    mapping = train_data.class_to_idx

    # Creating data indices for train and validation splits:
    data_size = len(train_data)
    indices = list(range(data_size))
    split = int(np.floor(validation_size * data_size))

    targets = train_data.targets
    train_idx, valid_idx = train_test_split(np.arange(len(targets)), 
                                            test_size=validation_size, 
                                            random_state=42, 
                                            shuffle=True, stratify=targets)

    train_sampler = sampler.SubsetRandomSampler(train_idx)
    valid_sampler = sampler.SubsetRandomSampler(valid_idx)

    train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
    validation_loader = DataLoader(valid_data, batch_size=batch_size, sampler=valid_sampler)

    print("Number of classes loaded: ", len(train_data.class_to_idx))
    print("Classes loaded: ", train_data.classes)

    print("Number of images loaded in training dataset: ", len(train_sampler))
    print("Number of images loaded in validation dataset: ", len(valid_sampler))
  
    print("Number of batches in training loader: ", len(train_loader))
    print("Number of batches in valuation loader: ", len(validation_loader))

    print("Finished Loading Train Data")
    print()

    return train_loader, validation_loader, len(train_sampler), len(valid_sampler), mapping

def LoadTestDataSet(test_dir, batch_size, test_transform):
    print("Loading Test Data ...")

    data = datasets.ImageFolder(root=test_dir, transform=test_transform)
    data_loader = DataLoader(data, batch_size=batch_size, shuffle=True)
    
    print("Number of classes loaded: ", len(data.classes))
    print("Classes loaded: ", data.classes)
    print("Number of images loaded: ", len(data))
    print("Finished Loading Test Data")
    print()

    return data_loader, data.classes


# Image transformations
trans = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    # Validation does not use augmentation
    'valid':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
}

In [0]:
trainloader, valloader = None, None
train_size, val_size = 0, 0
testloader = None

num_epoch = 10
num_classes = 25
device = "cuda" if torch.cuda.is_available() else "cpu" # Configure device
print(device)

dir = '/content/gdrive/Shared drives/eecs442-project/'
if num_classes == 10:
    train_dir = dir + 'arc10/arc10_train'
    test_dir = dir + '/arc10/arc10_test'
else: 
    train_dir = dir + 'arc25/arc25_train'
    test_dir = dir + 'arc25/arc25_test'

trainloader, valloader, train_size, val_size, mapping = LoadTrainDataSet(train_dir, 32, trans['train'], trans['valid'])
testloader, class_names = LoadTestDataSet(test_dir, 8, trans['valid'])

dataloaders = {'train': trainloader, 'val': valloader}
dataset_sizes = {'train':  train_size, 'val': val_size}

In [0]:
classes_train_data = {
  'Achaemenid architecture': 55,
  'American Foursquare architecture': 47,
  'American craftsman style': 156,
  'Ancient Egyptian architecture': 205,
  'Art Deco architecture': 293,
  'Art Nouveau architecture': 360,
  'Baroque architecture': 191,
  'Bauhaus architecture': 74,
  'Beaux-Arts architecture': 153,
  'Byzantine architecture': 89,
  'Chicago school architecture': 122,
  'Colonial architecture': 142,
  'Deconstructivism': 170,
  'Edwardian architecture': 63,
  'Georgian architecture': 123,
  'Gothic architecture': 87,
  'Greek Revival architecture': 262,
  'International style': 166,
  'Novelty architecture': 170,
  'Palladian architecture': 90,
  'Postmodern architecture': 130,
  'Queen Anne architecture': 340,
  'Romanesque architecture': 86,
  'Russian Revival architecture': 132,
  'Tudor Revival architecture': 130
}

labels = torch.zeros(25, dtype=torch.float)
for arcclass, num in classes_train_data.items(): 
  label = mapping[arcclass]
  labels[label] = num

weight = 1.0 / labels
weight = weight / torch.sum(weight)
weight = weight.to(device)

# **Class distribution in data sets**

In [0]:
idx2class_val = {v: 0 for _, v in mapping.items()}
for _, labels in valloader:
  for label in labels:
    idx2class_val[label.item()] += 1

In [0]:
before = [13, 10, 23, 46, 67, 64, 42, 14, 28, 13, 27, 21, 35, 12, 28, 11, 63, 39, 32, 16, 26, 73, 20, 23, 21]
after = [11, 9, 31, 41, 59, 72, 38, 15, 31, 18, 24, 29, 34, 13, 25, 17, 53, 33, 34, 18, 26, 68, 17, 26, 26]
original = [69,59,195,256,366,450,239,92,191,111,153,177,213,79,154, 109,327,207,212,113,163,425,107,165,162]

scaled = [int((768/4794)*val) for val in original]
print(scaled)

classes = [name[:-12] for name in class_names]

barWidth = 0.25
# Set position of bar on X axis
width = 0.35  # the width of the bars
x = np.arange(len(classes)) 

# Make the plot
plt.style.use('default')
fig, ax = plt.subplots(figsize=(10, 6))
plt.bar(x+0, before, width, color='tab:blue', label='Uniform Random Sampling')
plt.bar(x+0.25, after, width, color='tab:orange', label='Stratified Sampling')
plt.bar(x+0.5, scaled, width, color='tab:green', label='Actual Dataset, Scaled')
 
# Add xticks on the middle of the group bars
ax.set_ylabel('Number of Images')
ax.set_title('Class Distribution in Validation Set Before and After Stratified Sampling ')
ax.set_xticks(x+0.25)
ax.set_xticklabels(classes)
ax.legend()
plt.xticks(rotation='vertical')
fig.tight_layout()
plt.show()


# **Training and Evaluation Functions**

In [0]:
def train(model, trloader, valloader, criterion, optimizer, num_epoch = 10): # Train the model
    print("Start training...")
    model.train() # Set the model to training mode
    trloss = []
    valloss = []
    for i in range(num_epoch):
        running_loss = 0.0
        running_corrects = 0
        for batch, label in tqdm(trloader):
            batch = batch.to(device)
            label = label.to(device)
            optimizer.zero_grad() # Clear gradients from the previous iteration
            pred = model(batch) # This will call Network.forward() that you implement
            _, preds = torch.max(pred , 1)
            loss = criterion(pred, label) # Calculate the loss

            # statistics
            running_loss += loss.item() * batch.size(0)
            running_corrects += torch.sum(preds == label.data)

            loss.backward() # Backprop gradients to all tensors in the network
            optimizer.step() # Update trainable weights
        
        epoch_loss = running_loss / train_size
        epoch_acc = running_corrects.double() / train_size
        trloss.append(np.mean(epoch_loss))
        print('Epoch {}, Train Loss: {:.4f} Acc: {:.4f}'.format(i+1, epoch_loss, epoch_acc)) 
        
        running_loss = 0.0
        running_corrects = 0
        for batch, label in tqdm(valloader):
            batch = batch.to(device)
            label = label.to(device)
            with torch.no_grad():
                pred = model(batch) # This will call Network.forward() that you implement
                _, preds = torch.max(pred , 1)
                loss = criterion(pred, label) # Calculate the loss
                # statistics
                running_loss += loss.item() * batch.size(0)
                running_corrects += torch.sum(preds == label.data)

        epoch_loss = running_loss / val_size
        epoch_acc = running_corrects.double() / val_size
        valloss.append(np.mean(epoch_loss)) 
        print('Epoch {}, Val Loss: {:.4f} Acc: {:.4f}'.format(i+1, epoch_loss, epoch_acc))      

    epochs = np.arange(0, num_epoch, 1)
    plt.style.use('default')
    plt.plot(epochs, trloss, 'b', label='Training loss')
    plt.plot(epochs, valloss, 'r', label='Validation loss')
    plt.ylabel('Classification loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right', shadow=False, ncol=2)
    plt.show()
        
    print("Done!")

def evaluate(model, loader): # Evaluate accuracy on validation / test set
    model.eval() # Set the model to evaluation mode
    correct = 0
    with torch.no_grad(): # Do not calculate grident to speed up computation
        for batch, label in tqdm(loader):
            batch = batch.to(device)
            label = label.to(device)
            pred = model(batch)
            correct += (torch.argmax(pred,dim=1)==label).sum().item()
    acc = correct/len(loader.dataset)
    print("Evaluation accuracy: {}".format(acc))
    return acc


In [0]:
# this train_model function has been adapted from
# https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
# It accounts for learning rate decay scheduler that previous train funciton didn't

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    trloss = []
    valloss = []
    tracc = []
    valacc = []
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
  
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'train':
              trloss.append(epoch_loss)
              tracc.append(epoch_acc)
            else:
              valloss.append(epoch_loss)
              valacc.append(epoch_acc)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    epochs = np.arange(0, num_epochs, 1)
    plt.style.use('default')
    plt.subplot(2, 1, 1)
    plt.plot(epochs, trloss, 'b', label='Training loss')
    plt.plot(epochs, valloss, 'r', label='Validation loss')
    plt.ylabel('Classification loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right', shadow=False, ncol=2)

    plt.subplot(2, 1, 2)
    plt.plot(epochs, tracc, 'b', label='Training accuracy')
    plt.plot(epochs, valacc, 'r', label='Validation accuracy')
    plt.ylabel('Classification accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='lower right', shadow=False, ncol=2)

    plt.show()
        
    print("Done!")

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

def evaluate(model, loader): # Evaluate accuracy on validation / test set
    model.eval() # Set the model to evaluation mode
    correct = 0
    with torch.no_grad(): # Do not calculate grident to speed up computation
        for batch, label in tqdm(loader):
            batch = batch.to(device)
            label = label.to(device)
            pred = model(batch)
            correct += (torch.argmax(pred,dim=1)==label).sum().item()
    acc = correct/len(loader.dataset)
    print("Evaluation accuracy: {}".format(acc))
    return acc

# **ResNet50 with One Fully Connected Layer**

In [0]:
# Validation accuracy: 
# Test accuracy: 
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)
criterion = nn.CrossEntropyLoss(weight)

# Only parameters of final layer are being optimized
optimizer_conv = optim.SGD(model.fc.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-5)

# Decay LR by a factor of 0.7 every 5 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=5, gamma=0.7)

# num_epoch
model = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=25)

acc = evaluate(model, testloader)

# **ResNet50 with Custom Layers**

In [0]:
resnet50pre = models.resnet50(pretrained=True)
num_features = resnet50pre.fc.in_features
ofeatures = resnet50pre.fc.out_features
resnet50 = nn.Sequential(*list(resnet50pre.children())[:-2])

for param in resnet50.parameters():
   param.requires_grad = False

model = nn.Sequential(
    resnet50,
    nn.Conv2d(2048, 64, kernel_size=(1, 1), stride=(1, 1), bias=False),
    nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.Conv2d(64, 256,kernel_size=(1, 1), stride=(1, 1), bias=False),
    nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.Conv2d(256, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False),
    nn.BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(num_features, num_classes)
)

if torch.cuda.is_available():
    model.cuda()

# Hyperparameters after random search
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
start_time = time.time()
train(model, trainloader, valloader, 10) 
print('Training time: {:10f} minutes'.format((time.time()-start_time)/60))

print("Evaluate on validation set...")
evaluate(model, valloader)
print("Evaluate on test set")
evaluate(model, testloader)

# **Best ResNet50 Model with Custom Classifier**

In [0]:
# Validation accuracy: 69.05%
# Test accuracy: 65.03%
dataloaders = {'train': trainloader, 'val': valloader}
dataset_sizes = {'train':  train_size, 'val': val_size}

model_conv = models.resnet50(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Sequential(
    nn.Linear(num_ftrs, 1024), 
    nn.ReLU(inplace=True), 
    nn.Linear(1024, 256),
    nn.ReLU(inplace=True),
    nn.Linear(256, num_classes),
    nn.LogSoftmax(dim=1))

model_conv = model_conv.to(device)
criterion = nn.CrossEntropyLoss(weight)

# Hyperparameters after random search
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-5)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=5, gamma=0.7)
model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=25)

evaluate(model_conv, valloader)
evaluate(model_conv, testloader)

Save Model

In [0]:
torch.save(model_conv.state_dict(), '/content/gdrive/Shared drives/eecs442-project/resnet/saved_models/model_acc_69.05.pth')

# **Visualize Model**

Load Model

In [0]:
best_resnet = models.resnet50(pretrained=True)
for param in best_resnet.parameters():
    param.requires_grad = False

best_resnet.fc = nn.Sequential(
    nn.Linear(best_resnet.fc.in_features, 1024), 
    nn.ReLU(inplace=True), 
    nn.Linear(1024, 256),
    nn.ReLU(inplace=True),
    nn.Linear(256, num_classes),
    nn.LogSoftmax(dim=1))

best_resnet = best_resnet.to(device)
best_resnet.load_state_dict(torch.load('/content/gdrive/Shared drives/eecs442-project/resnet/saved_models/model_acc_69.05.pth'))

print(best_model)

In [0]:
from torchvision import utils

# this function has been adapted from the pytorch tutorial
# https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    plt.style.use('default')
    fig = plt.figure(figsize=(5,6))
  
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
   
            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                pred_class_name = class_names[preds[j]]
                act_class_name = class_names[labels[j]]
                ax.set_title('Prediction: \n {} \n Actual: \n {}'.format(pred_class_name, act_class_name), fontsize="8")
                image = inputs.cpu().data[j]
                image -= image.min()
                image /= image.max()
                plt.tight_layout()
                plt.imshow(utils.make_grid(image, nrow=5).permute(1, 2, 0))

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [0]:
visualize_model(best_resnet, num_images=4)

# **Load Data for InceptionV3**

In [0]:
# Image transformations
trans = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=300, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    # Validation does not use augmentation
    'valid':
    transforms.Compose([
        transforms.Resize(size=300),
        transforms.CenterCrop(size=299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
}

trainloader, valloader = None, None
train_size, val_size = 0, 0
testloader = None

num_epoch = 10
num_classes = 25
device = "cuda" if torch.cuda.is_available() else "cpu" # Configure device

dir = '/content/gdrive/Shared drives/eecs442-project/'
if num_classes == 10:
    train_dir = dir + 'arc10/arc10_train'
    test_dir = dir + '/arc10/arc10_test'
else: 
    train_dir = dir + 'arc25/arc25_train'
    test_dir = dir + 'arc25/arc25_test'

trainloader, valloader, train_size, val_size, mapping = LoadTrainDataSet(train_dir, 32, trans['train'], trans['valid'])
testloader, class_names = LoadTestDataSet(test_dir, 8, trans['valid'])

# **Evaluate InceptionV3 models**

In [0]:
# The InceptionV3 models were run in a seperate notebook
# Below is just an example of how the InceptionV3 models were trained and evaluated
# This code does include all the evalution we have done on InceptionV3

dataloaders = {'train': trainloader, 'val': valloader}
dataset_sizes = {'train':  train_size, 'val': val_size}

model_conv = models.inception_v3(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
num_ftrs2 = model_conv.AuxLogits.fc.in_features
# model_conv.fc = nn.Linear(num_ftrs, num_classes)
model_conv.fc = nn.Sequential(
    nn.Linear(num_ftrs, 1024), 
    nn.ReLU(inplace=True), 
    nn.Linear(1024, 256),
    nn.ReLU(inplace=True),
    nn.Linear(256, num_classes),
    nn.LogSoftmax(dim=1))
model_conv.AuxLogits.fc = nn.Sequential(
    nn.Linear(num_ftrs2, 1024), 
    nn.ReLU(inplace=True), 
    nn.Linear(1024, 256),
    nn.ReLU(inplace=True),
    nn.Linear(256, num_classes),
    nn.LogSoftmax(dim=1))

model_conv = model_conv.to(device)
criterion = nn.CrossEntropyLoss(weight)

# Hyperparameters after random search
optimizer_conv = optim.SGD(list(list(model_conv.fc.parameters()) + list(model_conv.AuxLogits.parameters())), lr=0.01, momentum=0.9, weight_decay=1e-5)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=5, gamma=0.7)
model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=20)

evaluate(model_conv, testloader)